Query from fake bank database stored in [data/db/fake_bank.db](data/db/fake_bank.db)

In [1]:
import os
from dotenv import load_dotenv

#openai
import openai

# ibm genai
from genai.credentials import Credentials
from genai.model import Model
from genai.schemas import GenerateParams, ModelType
from genai.extensions.langchain import LangChainInterface  # langchain support

# langchain
from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI

load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url)
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
# load model

"""
llm = LangChainInterface(
        model=ModelType.FLAN_UL2,  # choice of model is not necessarily optimal
        #model=ModelType.CODEGEN_MONO_16B,  # didn't work well
        credentials=Credentials(os.environ.get("GENAI_KEY")),
        params=GenerateParams(
            decoding_method="greedy",
            max_new_tokens=1024,
            min_new_tokens=1,
            repetition_penalty=2,
        ).dict()
    )
"""
llm = OpenAI(temperature=0)

In [3]:
# database connection and chain
db = SQLDatabase.from_uri("sqlite:///./data/db/fake_bank.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

db_chain = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,ersonal
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
db_chain.handle_parsing_errors = True

In [5]:
# correct
db_chain.run('how many unique clients are in the bank?')



> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: accounts, clients, transactions
Thought: I should query the schema of the clients table to see what columns I can use.
Action: sql_db_schema
Action Input: clients
Observation: 
CREATE TABLE clients (
	"Client_Name" TEXT, 
	"Client_Key" BIGINT
)

/*
3 rows from clients table:
Client_Name	Client_Key
richie mcrface	1234
robbie barker	2345
sum yung gai	3456
*/
Thought: I should query the Client_Key column to get the number of unique clients.
Action: sql_db_query
Action Input: SELECT COUNT(DISTINCT Client_Key) FROM clients
Observation: [(6,)]
Thought: I now know the final answer
Final Answer: There are 6 unique clients in the bank.

> Finished chain.


'There are 6 unique clients in the bank.'

In [6]:
# correct
db_chain.run('how many accounts are of the deposit product family in the bank?')



> Entering new  chain...
Thought: I should use the list_tables_sql_db tool to get a list of tables in the database.
Action: sql_db_list_tables
Action Input: ""
Observation: accounts, clients, transactions
Thought: I should query the schema of the accounts table to see what columns I can use.
Action: sql_db_schema
Action Input: "accounts"
Observation: 
CREATE TABLE accounts (
	"Account_Key" BIGINT, 
	"Client_key" BIGINT, 
	"Product_Family" TEXT, 
	"Account_Type" TEXT, 
	"Balance" FLOAT, 
	"Monthly_Average_Balance" FLOAT
)

/*
3 rows from accounts table:
Account_Key	Client_key	Product_Family	Account_Type	Balance	Monthly_Average_Balance
1	1234	deposit	checking	3450.23	45.12
2	1234	deposit	savings	13987.23	123980.0
3	1234	loan	multi-family-residential	1230000.0	None
*/
Thought: I should query the accounts table for the number of accounts of the deposit product family.
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM accounts WHERE Product_Family = 'deposit'
Observation: [(11,)]
Th

'There are 11 accounts of the deposit product family in the bank.'

In [7]:
# correct
db_chain.run('sum the dollars in the deposit product family accounts')



> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: accounts, clients, transactions
Thought: I should query the schema of the accounts table to see what columns I can use.
Action: sql_db_schema
Action Input: accounts
Observation: 
CREATE TABLE accounts (
	"Account_Key" BIGINT, 
	"Client_key" BIGINT, 
	"Product_Family" TEXT, 
	"Account_Type" TEXT, 
	"Balance" FLOAT, 
	"Monthly_Average_Balance" FLOAT
)

/*
3 rows from accounts table:
Account_Key	Client_key	Product_Family	Account_Type	Balance	Monthly_Average_Balance
1	1234	deposit	checking	3450.23	45.12
2	1234	deposit	savings	13987.23	123980.0
3	1234	loan	multi-family-residential	1230000.0	None
*/
Thought: I should query the sum of the balance column in the accounts table where the product family is deposit.
Action: sql_db_query
Action Input: SELECT SUM(Balance) FROM accounts WHERE Product_Family = 'deposit'
Observation: [(20197586.46,)]
Thought: I now know the final answer
Final Answer: The sum of the dollar

'The sum of the dollars in the deposit product family accounts is 20197586.46.'

In [8]:
# correct
db_chain.run('sum the monthly average balance in deposit accounts')



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: accounts, clients, transactions
Thought: I should query the schema of the accounts and transactions tables
Action: sql_db_schema
Action Input: "accounts, transactions"
Observation: 
CREATE TABLE accounts (
	"Account_Key" BIGINT, 
	"Client_key" BIGINT, 
	"Product_Family" TEXT, 
	"Account_Type" TEXT, 
	"Balance" FLOAT, 
	"Monthly_Average_Balance" FLOAT
)

/*
3 rows from accounts table:
Account_Key	Client_key	Product_Family	Account_Type	Balance	Monthly_Average_Balance
1	1234	deposit	checking	3450.23	45.12
2	1234	deposit	savings	13987.23	123980.0
3	1234	loan	multi-family-residential	1230000.0	None
*/


CREATE TABLE transactions (
	"Account_key" BIGINT, 
	"Client_key" BIGINT, 
	"Amount" BIGINT, 
	"Description" TEXT
)

/*
3 rows from transactions table:
Account_key	Client_key	Amount	Description
1	1234	-5	starbucks coffee
2	1234	-100	transfer to bob
1	1234	200	incoming direct deposit
*/
Thought: I should query

'The sum of the monthly average balance in deposit accounts is 20224834.48.'

In [9]:
db_chain.run('sum the balance and group by client where product family is deposit')



> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: accounts, clients, transactions
Thought: I should query the schema of the accounts and clients tables
Action: sql_db_schema
Action Input: accounts, clients
Observation: 
CREATE TABLE accounts (
	"Account_Key" BIGINT, 
	"Client_key" BIGINT, 
	"Product_Family" TEXT, 
	"Account_Type" TEXT, 
	"Balance" FLOAT, 
	"Monthly_Average_Balance" FLOAT
)

/*
3 rows from accounts table:
Account_Key	Client_key	Product_Family	Account_Type	Balance	Monthly_Average_Balance
1	1234	deposit	checking	3450.23	45.12
2	1234	deposit	savings	13987.23	123980.0
3	1234	loan	multi-family-residential	1230000.0	None
*/


CREATE TABLE clients (
	"Client_Name" TEXT, 
	"Client_Key" BIGINT
)

/*
3 rows from clients table:
Client_Name	Client_Key
richie mcrface	1234
robbie barker	2345
sum yung gai	3456
*/
Thought: I should query the accounts table for the sum of the balance and group by client where product family is deposit
Action: sql_db_query

'Richie Mcrface has a total balance of 17437.46, Robbie Barker has a total balance of 15740.0, Sum Yung Gai has a total balance of 3234.0, Frankie Columbo has a total balance of 178308.0, Arnie Gov has a total balance of 19907437.0, and Ira Beback has a total balance of 75430.0.'

In [13]:
html = db_chain.run('sum the balance and group by client for checking accounts. \
return the result as an html table of client name and balance. \
sort by balance in descending order')



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: accounts, clients, transactions
Thought: I should query the schema of the accounts and clients tables
Action: sql_db_schema
Action Input: "accounts, clients"
Observation: 
CREATE TABLE accounts (
	"Account_Key" BIGINT, 
	"Client_key" BIGINT, 
	"Product_Family" TEXT, 
	"Account_Type" TEXT, 
	"Balance" FLOAT, 
	"Monthly_Average_Balance" FLOAT
)

/*
3 rows from accounts table:
Account_Key	Client_key	Product_Family	Account_Type	Balance	Monthly_Average_Balance
1	1234	deposit	checking	3450.23	45.12
2	1234	deposit	savings	13987.23	123980.0
3	1234	loan	multi-family-residential	1230000.0	None
*/


CREATE TABLE clients (
	"Client_Name" TEXT, 
	"Client_Key" BIGINT
)

/*
3 rows from clients table:
Client_Name	Client_Key
richie mcrface	1234
robbie barker	2345
sum yung gai	3456
*/
Thought: I should query the accounts table to get the sum of the balance and group by client
Action: sql_db_query
Action Input: SELECT c.C

In [14]:
from IPython.core.display import HTML
HTML(html)

Client Name,Balance
arnie gov,9993450.0
frankie columbo,54321.0
robbie barker,15740.0
ira beback,6432.0
richie mcrface,3450.23
sum yung gai,1234.0


In [18]:
html = db_chain.run('look up all direct deposit transactions. show client name, \
amount as an html table, sorted by client name.')
HTML(html)



> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: accounts, clients, transactions
Thought: I should query the schema of the accounts and transactions tables.
Action: sql_db_schema
Action Input: accounts, transactions
Observation: 
CREATE TABLE accounts (
	"Account_Key" BIGINT, 
	"Client_key" BIGINT, 
	"Product_Family" TEXT, 
	"Account_Type" TEXT, 
	"Balance" FLOAT, 
	"Monthly_Average_Balance" FLOAT
)

/*
3 rows from accounts table:
Account_Key	Client_key	Product_Family	Account_Type	Balance	Monthly_Average_Balance
1	1234	deposit	checking	3450.23	45.12
2	1234	deposit	savings	13987.23	123980.0
3	1234	loan	multi-family-residential	1230000.0	None
*/


CREATE TABLE transactions (
	"Account_key" BIGINT, 
	"Client_key" BIGINT, 
	"Amount" BIGINT, 
	"Description" TEXT
)

/*
3 rows from transactions table:
Account_key	Client_key	Amount	Description
1	1234	-5	starbucks coffee
2	1234	-100	transfer to bob
1	1234	200	incoming direct deposit
*/
Thought: I should query th

Client Name,Amount
frankie columbo,987
richie mcrface,200
robbie barker,64
